## Template for plotting signal psd

In [ ]:
# Plotting Power Spectral Density of a channel in the eeg data before artifact removal

channel = 34
data_single = data_filtered[channel]

# Welch method parameters
fs = 128
window = 'hamming'
nperseg = 64

# Welch method
f, psd = welch(
    x= data_single,
    fs= fs,
    window= window,
    nperseg= nperseg,
)

trace = go.Scatter(
    x= f,
    y= psd,
    mode= 'lines',
)

data = [trace]

layout = go.Layout(
    title= 'PSD of channel {}'.format(channels[channel]),
    xaxis= dict(
        title= 'Frequency'
    ),
    yaxis=dict(
        title= 'PSD'
    )
)

fig = go.Figure(
    data= data,
    layout= layout,
)

fig.show()

## Template for plotting single channel eeg

In [ ]:
channel = 34
trace = go.Scatter(
    x=timesF, 
    y=data_filtered[channel], 
    mode='lines'
)

data = [trace]

layout = go.Layout(
    title=channels[channel],
    xaxis= dict(
        title= 'Time (s)'
    ),
    yaxis= dict(
        title= 'Amplitude'
    )
)

fig = go.Figure(
    data=data, 
    layout=layout
)

fig.show()

In [ ]:
import mne

# Assume the EEG data is loaded into a RawObject named raw

# Extract the data channel containing the EEG values
data = raw.get_data()

# The data shape will be (number_of_channels, number_of_time_points)
print(data.shape) # e.g. (64, 1000000)

# Compute the power spectral density (PSD) using the Welch method  
# with 1 second windows         
f, psd = mne.time_frequency.psd_welch(data, sfreq=raw.info['sfreq'], 
                                   n_per_seg = raw.info['sfreq'])  

# The PSD shape will be (number_of_frequencies, number_of_time_points)
print(psd.shape) # e.g. (2000, 50000)

# Integrate the PSD over the given frequency bands  
theta = np.sum(psd[:,f<8], axis=0)  
alpha = np.sum(psd[:,8<=f<12], axis=0)
low_beta = np.sum(psd[:,12<=f<16], axis=0)
high_beta = np.sum(psd[:,16<=f<30], axis=0)  
gamma = np.sum(psd[:,30<=f<44], axis=0)

# The band energies will have shape (number_of_time_points,)
print(theta.shape) # e.g. (50000,)

# Extract the band energies every 0.2 seconds    
features = np.vstack((theta[::5], 
                      alpha[::5], 
                      low_beta[::5],  
                      high_beta[::5], 
                      gamma[::5]))

# The final feature shape will be  
# (number_of_time_points, 5 frequency bands)           
print(features.shape) # e.g. (10000, 5)

In [8]:
import copy
import numpy as np
import math

Psd = [1, 2, math.nan, 4, math.nan, 5]

Psd = np.array(Psd)

psd_nonan = copy.copy(Psd)

psd_nonan = [x for x in psd_nonan if not np.isnan(x)]

display(Psd)

if np.any(np.isnan(Psd)):
    nonnan_values = Psd[~np.isnan(Psd)]
    nan_average = np.mean(nonnan_values) 
    Psd[np.isnan(Psd)] = nan_average

print(Psd) 
# [1, 2, nan, 4, nan, 5]

print(psd_nonan)
# [1, 2, 4, 5]

array([ 1.,  2., nan,  4., nan,  5.])

[1. 2. 3. 4. 3. 5.]
[1.0, 2.0, 4.0, 5.0]
